In [1]:
import psycopg2
from datetime import timedelta
from sqlalchemy import create_engine
import psycopg2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import psycopg2

conn = psycopg2.connect(
    host="localhost",
    database="mimic",
    user="postgres",
    password="postgres"
)

cur = conn.cursor()

cur.execute("SELECT version();")
print(cur.fetchone())

('PostgreSQL 15.2, compiled by Visual C++ build 1914, 64-bit',)


In [3]:
# Connect to db
conn = psycopg2.connect(host='localhost', dbname='mimic', user='postgres', password='postgres', options='-c search_path=mimiciii')
#conn = psycopg2.connect(dbname='mimic', user='postgres')
cur = conn.cursor() 

# Read in table with patients & admissions (inner join on subject_id) and icu_stays (inner joinon subject_id and hadm_id)
icustay_details = pd.read_sql_query("SELECT * FROM mimiciii.icustay_4days;", conn)

# Read in vital and lab signs
pivoted_vital = pd.read_sql_query("SELECT * FROM mimiciii.pivoted_vital_4days;", conn)
pivoted_lab = pd.read_sql_query("SELECT * FROM mimiciii.pivoted_lab_4days;", conn)

# Read in lab measurements
# Use flicu_pivoted_lab (as it only takes the lab tests during ICU stay)
#query = "SELECT * FROM mimiciii.flicu_pivoted_lab;"
# Alternative:  Use the lab values recorded previous to the ICU stay (although during same hospital admission!), 
# then sample them  (8h intervalls) and then forward fill plus cap at either icu admission time of first vital sign recorded
#query = "SELECT * FROM mimiciii.pivoted_lab;"
#pivoted_lab = pd.read_sql_query(query, conn)

# Close the cursor and connection to so the server can allocate bandwidth to other requests
cur.close()
conn.close()

C:\Users\aakru\anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\aakru\anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\aakru\anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


Inner join on lab and vitals with icustay_id and charttime 

In [4]:
vital = pivoted_vital.merge(pivoted_lab[['icustay_id', 'charttime']], on=['icustay_id', 'charttime'], how='inner').drop_duplicates()
print("Number of ICU stays in lab_filtered: ", vital['icustay_id'].nunique())
lab= pivoted_lab.merge(pivoted_vital[['icustay_id', 'charttime']], on=['icustay_id', 'charttime'], how='inner').drop_duplicates()
print("Number of ICU stays in lab_filtered: ", lab['icustay_id'].nunique())

Number of ICU stays in lab_filtered:  15711
Number of ICU stays in lab_filtered:  15711


In [17]:
lab[lab['icustay_id']==200003]

,icustay_id,subject_id,charttime,aniongap,albumin,bands,bicarbonate,bilirubin,creatinine,chloride,...,platelet,potassium,ptt,inr,pt,sodium,bun,wbc,bacteria,ckd
154208,200003,27513,2199-08-03 01:47:00,21.0,NaN,NaN,18.0,3.4,1.0,105.0,...,109.0,3.2,32.1,1.8,18.8,141.0,20.0,40.2,NaN,0
154209,200003,27513,2199-08-03 17:37:00,15.0,2.8,NaN,25.0,3.9,0.7,107.0,...,119.0,3.1,29.3,1.3,14.2,144.0,19.0,43.9,NaN,0
154210,200003,27513,2199-08-04 01:21:00,13.0,2.5,NaN,25.0,NaN,0.7,111.0,...,123.0,3.8,27.1,1.2,13.6,145.0,21.0,35.0,NaN,0
154211,200003,27513,2199-08-04 11:52:00,12.0,NaN,NaN,26.0,NaN,NaN,108.0,...,NaN,3.6,NaN,NaN,NaN,142.0,NaN,NaN,NaN,0
154212,200003,27513,2199-08-05 02:25:00,12.0,2.3,NaN,26.0,4.7,0.8,109.0,...,126.0,3.6,25.9,1.2,13.4,143.0,16.0,24.4,NaN,0
154213,200003,27513,2199-08-06 02:12:00,14.0,2.2,NaN,27.0,4.4,0.7,105.0,...,113.0,3.4,26.5,1.1,12.7,143.0,12.0,14.1,NaN,0
154214,200003,27513,2199-08-07 01:56:00,11.0,2.3,NaN,27.0,4.9,0.7,109.0,...,126.0,4.2,NaN,NaN,NaN,143.0,10.0,13.2,NaN,0
154215,200003,27513,2199-08-08 01:41:00,9.0,NaN,NaN,28.0,5.2,0.7,111.0,...,116.0,3.8,NaN,NaN,NaN,144.0,11.0,14.5,NaN,0


In [18]:
lab_4days = (
    lab.groupby('icustay_id')
    .apply(lambda x: x[x['charttime'] <= x.iloc[0]['charttime'] + pd.Timedelta(days=4, hours=1, minutes=0, seconds=0)])
    .reset_index(drop=True)
)

In [21]:
lab_4days

,icustay_id,subject_id,charttime,aniongap,albumin,bands,bicarbonate,bilirubin,creatinine,chloride,...,platelet,potassium,ptt,inr,pt,sodium,bun,wbc,bacteria,ckd
0,200003,27513,2199-08-03 01:47:00,21.0,NaN,NaN,18.0,3.4,1.0,105.0,...,109.0,3.2,32.1,1.8,18.8,141.0,20.0,40.2,NaN,0
1,200003,27513,2199-08-03 17:37:00,15.0,2.8,NaN,25.0,3.9,0.7,107.0,...,119.0,3.1,29.3,1.3,14.2,144.0,19.0,43.9,NaN,0
2,200003,27513,2199-08-04 01:21:00,13.0,2.5,NaN,25.0,NaN,0.7,111.0,...,123.0,3.8,27.1,1.2,13.6,145.0,21.0,35.0,NaN,0
3,200003,27513,2199-08-04 11:52:00,12.0,NaN,NaN,26.0,NaN,NaN,108.0,...,NaN,3.6,NaN,NaN,NaN,142.0,NaN,NaN,NaN,0
4,200003,27513,2199-08-05 02:25:00,12.0,2.3,NaN,26.0,4.7,0.8,109.0,...,126.0,3.6,25.9,1.2,13.4,143.0,16.0,24.4,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117299,299994,10718,2114-12-12 14:47:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
117300,299994,10718,2114-12-12 18:37:00,15.0,NaN,NaN,19.0,NaN,3.6,117.0,...,109.0,4.4,33.2,1.3,14.1,147.0,33.0,9.5,NaN,0
117301,299994,10718,2114-12-13 01:23:00,16.0,NaN,NaN,20.0,NaN,4.1,114.0,...,128.0,4.4,30.8,1.2,13.1,146.0,38.0,12.7,NaN,0
117302,299994,10718,2114-12-13 16:33:00,17.0,NaN,NaN,19.0,NaN,4.6,113.0,...,NaN,4.4,NaN,NaN,NaN,145.0,45.0,NaN,NaN,0


In [19]:
lab_4days[lab_4days['icustay_id']==200003]

,icustay_id,subject_id,charttime,aniongap,albumin,bands,bicarbonate,bilirubin,creatinine,chloride,...,platelet,potassium,ptt,inr,pt,sodium,bun,wbc,bacteria,ckd
0,200003,27513,2199-08-03 01:47:00,21.0,NaN,NaN,18.0,3.4,1.0,105.0,...,109.0,3.2,32.1,1.8,18.8,141.0,20.0,40.2,NaN,0
1,200003,27513,2199-08-03 17:37:00,15.0,2.8,NaN,25.0,3.9,0.7,107.0,...,119.0,3.1,29.3,1.3,14.2,144.0,19.0,43.9,NaN,0
2,200003,27513,2199-08-04 01:21:00,13.0,2.5,NaN,25.0,NaN,0.7,111.0,...,123.0,3.8,27.1,1.2,13.6,145.0,21.0,35.0,NaN,0
3,200003,27513,2199-08-04 11:52:00,12.0,NaN,NaN,26.0,NaN,NaN,108.0,...,NaN,3.6,NaN,NaN,NaN,142.0,NaN,NaN,NaN,0
4,200003,27513,2199-08-05 02:25:00,12.0,2.3,NaN,26.0,4.7,0.8,109.0,...,126.0,3.6,25.9,1.2,13.4,143.0,16.0,24.4,NaN,0
5,200003,27513,2199-08-06 02:12:00,14.0,2.2,NaN,27.0,4.4,0.7,105.0,...,113.0,3.4,26.5,1.1,12.7,143.0,12.0,14.1,NaN,0
6,200003,27513,2199-08-07 01:56:00,11.0,2.3,NaN,27.0,4.9,0.7,109.0,...,126.0,4.2,NaN,NaN,NaN,143.0,10.0,13.2,NaN,0


In [22]:
print(lab_4days[lab_4days['icustay_id']==299994]['charttime'].max()- lab_4days[lab_4days['icustay_id']==299994]['charttime'].min()) 

4 days 00:56:00


In [27]:
count = 0
for icustay_id in lab_4days['icustay_id'].unique():
    diff = lab_4days[lab_4days['icustay_id']==icustay_id]['charttime'].max() - lab_4days[lab_4days['icustay_id']==icustay_id]['charttime'].min()
    if diff < pd.Timedelta(days=4):
        count += 1
        print(diff)
print(count)

3 days 06:30:00
3 days 11:00:00
3 days 14:00:00
2 days 00:00:00
3 days 20:00:00
3 days 05:05:00
0 days 00:00:00
3 days 18:11:00
2 days 02:00:00
0 days 00:00:00
1 days 04:30:00
0 days 00:00:00
3 days 22:26:00
1 days 00:00:00
2 days 23:32:00
1 days 15:03:00
0 days 00:00:00
2 days 23:17:00
3 days 02:41:00
2 days 23:46:00
0 days 00:00:00
3 days 00:00:00
1 days 14:46:00
3 days 04:50:00
3 days 15:39:00
3 days 19:00:00
3 days 10:35:00
0 days 00:00:00
2 days 23:30:00
3 days 14:27:00
1 days 06:00:00
3 days 19:06:00
3 days 10:31:00
3 days 18:08:00
0 days 00:00:00
0 days 00:00:00
3 days 08:47:00
2 days 22:59:00
3 days 12:18:00
2 days 06:15:00
3 days 04:43:00
3 days 09:23:00
3 days 23:08:00
3 days 23:14:00
3 days 23:55:00
3 days 17:00:00
3 days 13:46:00
3 days 11:35:00
0 days 00:00:00
3 days 11:46:00
3 days 05:37:00
3 days 19:00:00
3 days 20:45:00
3 days 11:27:00
3 days 12:30:00
3 days 23:13:00
0 days 00:00:00
3 days 12:52:00
3 days 10:29:00
3 days 14:01:00
3 days 07:32:00
3 days 00:31:00
3 days 1

3 days 22:07:00
3 days 13:10:00
0 days 01:20:00
3 days 21:28:00
0 days 00:00:00
0 days 00:00:00
3 days 23:30:00
3 days 11:00:00
3 days 07:16:00
3 days 09:57:00
3 days 23:50:00
3 days 08:28:00
3 days 07:02:00
3 days 23:45:00
2 days 11:00:00
3 days 12:43:00
0 days 21:30:00
3 days 19:56:00
3 days 23:30:00
0 days 00:00:00
2 days 16:15:00
0 days 00:00:00
0 days 00:00:00
3 days 08:13:00
2 days 23:15:00
2 days 00:00:00
3 days 08:46:00
3 days 14:01:00
3 days 18:58:00
0 days 00:00:00
3 days 19:29:00
3 days 20:39:00
3 days 22:45:00
0 days 00:00:00
1 days 06:42:00
3 days 14:01:00
2 days 13:27:00
3 days 12:26:00
2 days 23:24:00
0 days 00:00:00
3 days 23:46:00
3 days 04:57:00
3 days 22:36:00
3 days 19:06:00
0 days 07:00:00
3 days 23:08:00
0 days 00:00:00
3 days 08:00:00
3 days 16:45:00
3 days 11:15:00
3 days 12:00:00
0 days 00:00:00
3 days 21:30:00
3 days 13:21:00
3 days 08:45:00
3 days 08:53:00
3 days 19:33:00
3 days 16:32:00
3 days 18:00:00
3 days 22:00:00
0 days 00:00:00
3 days 19:04:00
3 days 1

3 days 18:00:00
3 days 10:14:00
3 days 10:16:00
0 days 00:00:00
3 days 22:45:00
3 days 19:45:00
3 days 19:11:00
0 days 00:00:00
3 days 00:08:00
3 days 01:17:00
3 days 04:39:00
3 days 07:42:00
3 days 05:20:00
3 days 22:10:00
3 days 14:18:00
3 days 11:08:00
3 days 15:14:00
3 days 04:34:00
3 days 03:00:00
1 days 14:35:00
0 days 00:00:00
3 days 07:15:00
3 days 22:10:00
3 days 04:52:00
3 days 03:20:00
1 days 17:20:00
3 days 16:11:00
0 days 00:00:00
3 days 05:57:00
3 days 19:24:00
2 days 09:00:00
3 days 00:00:00
3 days 11:09:00
3 days 16:40:00
3 days 02:00:00
2 days 01:08:00
1 days 19:00:00
3 days 18:11:00
3 days 09:00:00
0 days 00:05:00
1 days 23:00:00
3 days 07:42:00
0 days 00:00:00
0 days 00:00:00
3 days 00:00:00
3 days 04:57:00
3 days 06:19:00
2 days 22:45:00
3 days 11:56:00
3 days 14:45:00
3 days 23:35:00
1 days 08:45:00
2 days 21:29:00
3 days 15:26:00
3 days 09:50:00
1 days 15:00:00
1 days 15:59:00
3 days 01:30:00
3 days 08:06:00
3 days 22:06:00
3 days 12:28:00
0 days 00:00:00
0 days 0

2 days 21:24:00
3 days 09:00:00
3 days 04:22:00
3 days 08:02:00
2 days 15:45:00
3 days 16:50:00
0 days 17:58:00
3 days 21:11:00
3 days 20:27:00
3 days 11:34:00
3 days 01:00:00
0 days 23:00:00
0 days 00:00:00
0 days 00:00:00
0 days 00:00:00
3 days 02:05:00
3 days 11:56:00
3 days 05:47:00
0 days 00:00:00
0 days 00:00:00
0 days 00:00:00
3 days 07:14:00
3 days 02:51:00
3 days 07:50:00
3 days 23:09:00
3 days 12:00:00
3 days 13:15:00
3 days 20:21:00
3 days 04:44:00
0 days 00:00:00
0 days 00:00:00
2 days 22:01:00
3 days 22:26:00
0 days 00:00:00
3 days 10:33:00
3 days 10:45:00
0 days 00:00:00
3 days 19:11:00
2 days 04:35:00
3 days 07:29:00
3 days 22:54:00
3 days 18:38:00
0 days 00:00:00
3 days 11:30:00
3 days 05:15:00
3 days 01:36:00
3 days 13:10:00
3 days 23:05:00
3 days 23:56:00
3 days 20:15:00
3 days 23:21:00
0 days 23:53:00
3 days 05:52:00
3 days 03:52:00
3 days 11:18:00
3 days 15:19:00
3 days 21:34:00
3 days 23:31:00
3 days 14:13:00
3 days 12:06:00
3 days 06:00:00
3 days 12:17:00
3 days 0

3 days 10:27:00
3 days 12:20:00
3 days 23:07:00
2 days 23:58:00
3 days 17:47:00
3 days 05:00:00
3 days 23:16:00
3 days 13:05:00
3 days 09:00:00
3 days 10:04:00
3 days 03:37:00
3 days 20:56:00
2 days 22:30:00
0 days 00:00:00
0 days 20:00:00
2 days 00:13:00
3 days 17:16:00
1 days 04:30:00
3 days 17:30:00
3 days 22:48:00
3 days 01:32:00
3 days 13:50:00
3 days 08:00:00
3 days 15:46:00
2 days 16:47:00
2 days 04:30:00
3 days 23:40:00
3 days 15:09:00
1 days 20:00:00
3 days 18:20:00
3 days 09:00:00
3 days 22:00:00
3 days 12:05:00
0 days 00:00:00
0 days 00:00:00
3 days 12:46:00
3 days 22:53:00
3 days 21:45:00
3 days 23:36:00
3 days 22:40:00
3 days 19:15:00
2 days 20:30:00
3 days 22:08:00
0 days 00:00:00
2 days 23:56:00
2 days 07:00:00
2 days 18:00:00
3 days 11:50:00
3 days 06:15:00
3 days 23:56:00
3 days 03:17:00
3 days 11:18:00
3 days 23:57:00
3 days 11:20:00
0 days 00:00:00
2 days 17:00:00
3 days 10:07:00
0 days 00:00:00
3 days 21:40:00
0 days 00:00:00
3 days 22:11:00
1 days 03:00:00
0 days 1

...

3 days 13:30:00
0 days 00:00:00
0 days 00:00:00
3 days 18:58:00
0 days 00:00:00
2 days 10:00:00
1 days 12:15:00
0 days 00:00:00
0 days 00:00:00
2 days 10:57:00
3 days 17:05:00
3 days 12:51:00
0 days 00:00:00
0 days 00:00:00
3 days 23:33:00
0 days 00:00:00
2 days 18:15:00
3 days 23:25:00
2 days 20:20:00
3 days 21:43:00
3 days 11:22:00
3 days 06:51:00
3 days 14:24:00
0 days 00:00:00
3 days 20:44:00
2 days 09:26:00
2 days 04:00:00
3 days 21:34:00
1 days 07:15:00
3 days 11:54:00
2 days 23:42:00
2 days 04:00:00
3 days 04:26:00
1 days 19:56:00
2 days 10:00:00
3 days 04:42:00
3 days 23:49:00
3 days 19:30:00
0 days 00:00:00
3 days 23:08:00
0 days 00:00:00
0 days 14:00:00
3 days 22:59:00
1 days 09:05:00
2 days 16:25:00
3 days 12:16:00
3 days 09:15:00
3 days 22:12:00
3 days 20:43:00
3 days 20:00:00
3 days 02:48:00
3 days 23:28:00
3 days 16:03:00
3 days 17:00:00
3 days 04:00:00
0 days 21:00:00
3 days 01:17:00
3 days 21:20:00
3 days 22:54:00
1 days 05:00:00
3 days 15:00:00
0 days 00:00:00
3 days 1

3 days 12:43:00
2 days 16:00:00
3 days 17:09:00
3 days 20:39:00
2 days 00:31:00
0 days 12:00:00
3 days 07:08:00
0 days 00:00:00
3 days 16:16:00
3 days 20:11:00
3 days 21:00:00
3 days 08:52:00
3 days 06:30:00
3 days 22:45:00
3 days 01:24:00
3 days 18:24:00
3 days 13:45:00
1 days 02:00:00
3 days 15:50:00
3 days 18:39:00
3 days 08:45:00
3 days 01:00:00
3 days 04:32:00
3 days 17:42:00
1 days 10:01:00
3 days 11:17:00
1 days 23:00:00
2 days 00:00:00
3 days 22:03:00
1 days 14:00:00
3 days 23:59:00
3 days 23:28:00
3 days 11:40:00
3 days 18:43:00
2 days 06:00:00
3 days 23:04:00
1 days 03:00:00
3 days 12:52:00
3 days 07:34:00
3 days 19:20:00
3 days 13:23:00
2 days 12:00:00
3 days 10:49:00
3 days 14:02:00
3 days 10:07:00
1 days 10:30:00
3 days 21:43:00
3 days 02:58:00
3 days 05:16:00
3 days 18:54:00
3 days 08:56:00
1 days 08:00:00
0 days 00:00:00
0 days 22:30:00
0 days 00:00:00
3 days 02:15:00
3 days 12:50:00
3 days 21:31:00
3 days 22:54:00
3 days 12:04:00
3 days 23:30:00
3 days 23:42:00
0 days 2

3 days 22:50:00
3 days 23:00:00
3 days 15:55:00
3 days 11:50:00
3 days 10:59:00
2 days 15:10:00
3 days 04:00:00
3 days 21:32:00
3 days 11:17:00
3 days 14:16:00
1 days 14:37:00
3 days 22:19:00
0 days 00:00:00
1 days 13:30:00
2 days 00:00:00
3 days 11:35:00
0 days 00:00:00
0 days 00:00:00
3 days 02:11:00
1 days 03:25:00
3 days 12:59:00
3 days 20:13:00
2 days 22:33:00
3 days 10:26:00
3 days 00:36:00
3 days 18:58:00
3 days 14:31:00
3 days 14:18:00
3 days 18:22:00
3 days 04:10:00
0 days 00:00:00
2 days 10:00:00
3 days 23:20:00
0 days 00:00:00
3 days 20:45:00
3 days 20:26:00
2 days 00:51:00
2 days 10:30:00
3 days 22:26:00
3 days 23:00:00
3 days 22:17:00
0 days 00:00:00
3 days 17:46:00
3 days 15:19:00
1 days 07:15:00
3 days 10:35:00
0 days 00:00:00
2 days 15:19:00
0 days 08:00:00
3 days 14:46:00
3 days 08:23:00
2 days 23:18:00
0 days 00:00:00
2 days 04:00:00
0 days 13:37:00
0 days 20:24:00
0 days 00:00:00
2 days 22:00:00
0 days 19:59:00
3 days 13:00:00
3 days 04:03:00
2 days 22:28:00
0 days 0

0 days 00:00:00
3 days 20:00:00
2 days 08:45:00
3 days 02:51:00
3 days 04:47:00
3 days 20:01:00
3 days 08:42:00
3 days 21:12:00
3 days 11:15:00
3 days 06:21:00
1 days 15:00:00
3 days 23:21:00
3 days 23:34:00
3 days 07:45:00
3 days 20:06:00
0 days 00:00:00
3 days 21:36:00
3 days 22:37:00
0 days 22:00:00
3 days 18:50:00
3 days 14:09:00
3 days 01:28:00
3 days 11:25:00
3 days 05:04:00
3 days 22:30:00
0 days 00:00:00
3 days 14:04:00
3 days 13:00:00
1 days 20:00:00
3 days 05:00:00
2 days 22:58:00
2 days 14:00:00
0 days 00:00:00
2 days 18:50:00
3 days 17:33:00
3 days 21:28:00
0 days 00:00:00
2 days 10:00:00
3 days 23:54:00
3 days 13:17:00
0 days 00:00:00
3 days 04:35:00
3 days 13:27:00
0 days 17:15:00
1 days 23:01:00
1 days 17:45:00
3 days 07:00:00
2 days 14:10:00
0 days 20:20:00
3 days 04:14:00
3 days 12:56:00
3 days 23:06:00
3 days 09:44:00
3 days 07:14:00
3 days 06:08:00
3 days 22:00:00
3 days 23:26:00
3 days 02:45:00
0 days 23:56:00
0 days 00:00:00
3 days 19:16:00
1 days 12:00:00
3 days 1

3 days 10:14:00
3 days 11:59:00
3 days 00:25:00
3 days 22:22:00
3 days 13:37:00
0 days 00:00:00
3 days 05:44:00
0 days 00:00:00
3 days 23:13:00
3 days 06:23:00
2 days 23:22:00
3 days 12:55:00
2 days 00:54:00
3 days 05:41:00
3 days 06:31:00
3 days 16:56:00
2 days 02:05:00
3 days 14:22:00
3 days 17:57:00
3 days 16:55:00
3 days 23:32:00
3 days 22:44:00
3 days 10:39:00
3 days 01:16:00
1 days 05:50:00
0 days 02:01:00
3 days 12:00:00
3 days 22:10:00
3 days 14:27:00
3 days 17:30:00
3 days 13:08:00
1 days 05:00:00
3 days 21:12:00
3 days 19:49:00
3 days 23:55:00
3 days 13:42:00
3 days 17:49:00
3 days 00:10:00
3 days 08:59:00
3 days 23:00:00
3 days 10:12:00
3 days 18:20:00
3 days 08:31:00
3 days 10:46:00
3 days 12:45:00
3 days 13:57:00
3 days 03:09:00
0 days 00:00:00
0 days 19:50:00
3 days 12:38:00
3 days 17:21:00
3 days 20:25:00
3 days 23:43:00
3 days 20:02:00
1 days 04:00:00
0 days 07:15:00
3 days 10:58:00
3 days 02:13:00
3 days 17:39:00
3 days 23:58:00
0 days 19:45:00
0 days 00:00:00
3 days 1

In [30]:
count = 0
for icustay_id in lab_4days['icustay_id'].unique():
    diff = lab_4days[lab_4days['icustay_id']==icustay_id]['charttime'].max() - lab_4days[lab_4days['icustay_id']==icustay_id]['charttime'].min()
    if diff == pd.Timedelta(days=4):
        count += 1
print(count)

73


In [35]:
count = 0
for icustay_id in lab_4days['icustay_id'].unique():
    diff = lab_4days[lab_4days['icustay_id']==icustay_id]['charttime'].max() - lab_4days[lab_4days['icustay_id']==icustay_id]['charttime'].min()
    if diff >= pd.Timedelta(days=4) and  diff < pd.Timedelta(days=4, hours=5):
        count += 1
        #print(diff)
print(count)

1194


In [26]:
 len(lab_4days['icustay_id'].unique()) -13546

2165

In [5]:
vital_resampled = vital.copy()

# Resample from the end of the time series (how="last")
vital_resampled = vital_resampled.assign(charttime=vital_resampled.charttime.dt.round('H'))
#vital_resampled = vital_resampled.set_index('charttime').groupby('icustay_id').resample('1H', origin="end").median().drop(['icustay_id'], axis = 1).reset_index()
# Resample from the beginning of the time series
vital_resampled = vital_resampled.set_index('charttime').groupby('icustay_id').resample('1H', origin="start").median().drop(['icustay_id'], axis = 1).reset_index()

# Forward and backwards fill (use lambda function instead of directly applying it to groupby otherwise results from one group are carreid forward to another group...BAD)
# Fill NaNs (-1)
vital_col = vital_resampled.columns.drop(['icustay_id', 'charttime'])
vital_resampled = vital_resampled.set_index(['icustay_id', 'charttime']).groupby('icustay_id')[vital_col].transform(lambda x: x.ffill().bfill()).fillna(-1).reset_index()
#.fillna(value=vital_resampled[vital_columns].mean()) #.fillna(0)#.fillna(-1)

In [6]:
lab_resampled = lab.copy()
# Cut out minutes and hours, so that the resampling of the 8h takes the same time span as the 1h samples (for vitals)
lab_resampled = lab_resampled.assign(charttime=lab_resampled.charttime.dt.round('H'))
# Resample from the end of the time series 
#lab_resampled = lab_resampled.set_index('charttime').groupby('icustay_id').resample('8h', origin="end").median().drop(['icustay_id'], axis = 1).reset_index()
lab_resampled = lab_resampled.set_index('charttime').groupby('icustay_id').resample('8h', origin="start").median().drop(['icustay_id'], axis = 1).reset_index()

# Forward and backwards fill (use transform instead of direct groupby otherwise results from one group are carreid forward to another group...BAD)
# Fill NaNs (-1 or 0 or mean!?)
lab_col = lab_resampled.columns.drop(['icustay_id', 'charttime'])
lab_resampled = lab_resampled.set_index(['icustay_id', 'charttime']).groupby('icustay_id')[lab_col].transform(lambda x: x.ffill().bfill()).fillna(-1).reset_index()


In [210]:
icustay_4days = icustay_details[(icustay_details['outtime'] - icustay_details['intime']).dt.floor('d') >= pd.Timedelta(hours=96)]

lab_4days = lab_resampled.merge(icustay_4days[['icustay_id', 'intime']], on='icustay_id',how='inner')
lab_4days = lab_4days.sort_values(by=['icustay_id', 'charttime']) # sort by icustay_id and charttime
lab_4days['time_diff'] = lab_4days.groupby('icustay_id')['charttime'].diff() # calculate time difference between consecutive charttime values for each icustay_id
lab_4days = lab_4days[ (lab_4days['time_diff'].dt.days <5)]

In [11]:
print(lab_4days[lab_4days['icustay_id']==200003.0]['charttime'].max()- lab_4days[lab_4days['icustay_id']==200003.0]['charttime'].min()) 

3 days 00:25:00


In [205]:
count = 0
for icustay_id in lab_4days['icustay_id'].unique():
    diff = lab_4days[lab_4days['icustay_id']==icustay_id]['charttime'].max() - lab_4days[lab_4days['icustay_id']==icustay_id]['charttime'].min()
    if diff < pd.Timedelta(days=4):
        count += 1
print(count)

KeyboardInterrupt: 

In [212]:
for icu_id in lab_4days['icustay_id'].unique():
    icu_data = lab_4days[lab_4days['icustay_id'] == icu_id]
    diff = icu_data['charttime'].max() - icu_data['charttime'].min()
    print(f"ICU stay ID {icu_id}: {diff}")

ICU stay ID 200003.0: 4 days 16:00:00
ICU stay ID 200017.0: 2 days 16:00:00
ICU stay ID 200030.0: 5 days 00:00:00
ICU stay ID 200033.0: 3 days 00:00:00
ICU stay ID 200037.0: 1 days 16:00:00
ICU stay ID 200039.0: 5 days 16:00:00
ICU stay ID 200045.0: 18 days 16:00:00
ICU stay ID 200046.0: 51 days 16:00:00
ICU stay ID 200059.0: 12 days 08:00:00
ICU stay ID 200060.0: 1 days 16:00:00
ICU stay ID 200063.0: 13 days 00:00:00
ICU stay ID 200065.0: 12 days 16:00:00
ICU stay ID 200074.0: 5 days 16:00:00
ICU stay ID 200077.0: 14 days 16:00:00
ICU stay ID 200091.0: 3 days 08:00:00
ICU stay ID 200096.0: 0 days 16:00:00
ICU stay ID 200098.0: 2 days 08:00:00
ICU stay ID 200102.0: 7 days 00:00:00
ICU stay ID 200103.0: 7 days 00:00:00
ICU stay ID 200109.0: 9 days 08:00:00
ICU stay ID 200114.0: 6 days 08:00:00
ICU stay ID 200124.0: 3 days 16:00:00
ICU stay ID 200125.0: 2 days 08:00:00
ICU stay ID 200131.0: 6 days 16:00:00
ICU stay ID 200136.0: 2 days 08:00:00
ICU stay ID 200140.0: 19 days 08:00:00
ICU s

ICU stay ID 201517.0: 4 days 16:00:00
ICU stay ID 201523.0: 3 days 16:00:00
ICU stay ID 201525.0: 7 days 08:00:00
ICU stay ID 201528.0: 15 days 16:00:00
ICU stay ID 201537.0: 9 days 16:00:00
ICU stay ID 201545.0: 24 days 08:00:00
ICU stay ID 201546.0: 11 days 16:00:00
ICU stay ID 201555.0: 10 days 08:00:00
ICU stay ID 201556.0: 13 days 00:00:00
ICU stay ID 201569.0: 3 days 00:00:00
ICU stay ID 201578.0: 8 days 00:00:00
ICU stay ID 201580.0: 2 days 16:00:00
ICU stay ID 201587.0: 0 days 00:00:00
ICU stay ID 201588.0: 21 days 16:00:00
ICU stay ID 201596.0: 5 days 16:00:00
ICU stay ID 201608.0: 4 days 00:00:00
ICU stay ID 201611.0: 39 days 16:00:00
ICU stay ID 201612.0: 13 days 16:00:00
ICU stay ID 201614.0: 15 days 08:00:00
ICU stay ID 201625.0: 4 days 00:00:00
ICU stay ID 201627.0: 4 days 08:00:00
ICU stay ID 201641.0: 7 days 08:00:00
ICU stay ID 201648.0: 3 days 00:00:00
ICU stay ID 201649.0: 4 days 00:00:00
ICU stay ID 201652.0: 6 days 16:00:00
ICU stay ID 201664.0: 3 days 08:00:00
ICU

ICU stay ID 202995.0: 4 days 08:00:00
ICU stay ID 202996.0: 20 days 16:00:00
ICU stay ID 203005.0: 3 days 00:00:00
ICU stay ID 203011.0: 5 days 16:00:00
ICU stay ID 203013.0: 2 days 00:00:00
ICU stay ID 203015.0: 8 days 16:00:00
ICU stay ID 203025.0: 31 days 16:00:00
ICU stay ID 203028.0: 4 days 16:00:00
ICU stay ID 203034.0: 3 days 00:00:00
ICU stay ID 203060.0: 2 days 16:00:00
ICU stay ID 203061.0: 5 days 00:00:00
ICU stay ID 203062.0: 4 days 08:00:00
ICU stay ID 203063.0: 6 days 16:00:00
ICU stay ID 203076.0: 40 days 08:00:00
ICU stay ID 203082.0: 11 days 00:00:00
ICU stay ID 203093.0: 13 days 08:00:00
ICU stay ID 203104.0: 4 days 00:00:00
ICU stay ID 203105.0: 6 days 16:00:00
ICU stay ID 203106.0: 30 days 16:00:00
ICU stay ID 203108.0: 6 days 16:00:00
ICU stay ID 203111.0: 6 days 08:00:00
ICU stay ID 203121.0: 59 days 00:00:00
ICU stay ID 203134.0: 3 days 08:00:00
ICU stay ID 203138.0: 1 days 08:00:00
ICU stay ID 203142.0: 2 days 16:00:00
ICU stay ID 203143.0: 47 days 08:00:00
ICU 

KeyboardInterrupt: 